## Chronic Kidney Disease 

Chronic kidney disease (CKD) means your kidneys are damaged and can't filter blood the way they should. The disease is called “chronic” because the damage to your kidneys happens slowly over a long period of time. This damage can cause wastes to build up in your body. CKD can also cause other health problems.

<img src="https://medicaldialogues.in/h-upload/2020/12/30/500x300_145030-chronic-kidney-disease.webp" width="800px">

####  Description
The data was taken over a 2-month period in India with 25 features ( eg, red blood cell count, white blood cell count, etc). The target is the 'classification', which is either 'ckd' or 'notckd' - ckd=chronic kidney disease. Use machine learning techniques to predict if a patient is suffering from a chronic kidney disease or not.


#### Data Set Information:

We use the following representation to collect the dataset:-

* age - age
* bp - blood pressure
* sg - specific gravity
* al - albumin
* su - sugar
* rbc - red blood cells
* pc - pus cell
* pcc - pus cell clumps
* ba - bacteria
* bgr - blood glucose random
* bu - blood urea
* sc - serum creatinine
* sod - sodium
* pot - potassium
* hemo - hemoglobin
* pcv - packed cell volume
* wc - white blood cell count
* rc - red blood cell count
* htn - hypertension
* dm - diabetes mellitus
* cad - coronary artery disease
* appet - appetite
* pe - pedal edema
* ane - anemia
* class - class


#### Dataset Link 

[Here ](https://www.kaggle.com/colearninglounge/chronic-kidney-disease)


In [ ]:
#%cd ../
#!mkdir tmp
#%cd tmp

In [ ]:
#%cd tmp//kidney_disease/

In [ ]:
# Install W&B 
#!pip install -q --upgrade wandb
# Login 
#import wandb
#wandb.login()

In [ ]:
#Import Libraries
import glob
from keras.models import Sequential, load_model
import numpy as np
import pandas as pd
import seaborn as sns
import keras as k
from keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping
from tensorflow.keras.optimizers import Adam

In [ ]:
df_train = pd.read_csv('../input/chronic-kidney-disease/kidney_disease_train.csv')

df_test = pd.read_csv('../input/chronic-kidney-disease/kidney_disease_test.csv')

In [ ]:
df_train

In [ ]:
df = pd.concat([df_train, df_test])
df

In [ ]:
# Generate the Profiling Report

import pandas_profiling as pp
profile = pp.ProfileReport(
    df, title="Chronic Kidney Disease", html={"style": {"full_width": True}}, sort="None"
)

In [ ]:
profile

In [ ]:
df.info()

In [ ]:
df.shape

In [ ]:
# Initialize a new run
#wandb.init(project="visualize-predictions", name="metrics")

# Log the metric on each step
#for dfn in df['classification']:
    #wandb.log({"pcv": dfn})

#wandb.finish()


##   Run summary visit the link
## https://wandb.ai/yasirhessein/visualize-predictions/runs/2g0bsk7k

In [ ]:
# Initialize a new run
#wandb.init(project="visualize-predictions", name="metrics")

# Log the metric on each step
#for dfn in df['classification']:
    #wandb.log({"htn": dfn})

#wandb.finish()

## Run summary visit the link
## https://wandb.ai/yasirhessein/visualize-predictions/runs/1uw65vea

In [ ]:
#Create a list of columns to retain
retain = ["sg", "age","bp","al", "su","sc", "hemo","pcv", "wbcc", "rbcc", "htn", "classification"]

#retain = df.columns, Drop the columns that are not in retain
df = df.drop([col for col in df.columns if not col in retain], axis=1)
    
# Drop the rows with na or missing values
df = df.dropna(axis=0)

In [ ]:
def check_df(dataframe, head=5):
    print("##################### Shape #####################")
    print(dataframe.shape)
    print("##################### Types #####################")
    print(dataframe.dtypes)
    print("##################### Head #####################")
    print(dataframe.head(head))
    print("##################### Tail #####################")
    print(dataframe.tail(head))
    print("##################### NA #####################")
    print(dataframe.isnull().sum())
check_df(df)

In [ ]:
sns.countplot(df['classification'],label="Count")

In [ ]:
plt.figure(figsize=(20,10))  #This is used to change the size of the figure/ heatmap
sns.heatmap(df.corr(), annot=True, fmt='.0%')

In [ ]:
print(retain)

In [ ]:
#Transform non-numeric columns into numerical columns
for column in df.columns:
        if df[column].dtype == np.number:
            continue
        df[column] = LabelEncoder().fit_transform(df[column])

In [ ]:
#Split the data
x= df[['sg', 'al', 'sc', 'pcv', 'htn']]
y = df['classification']

In [ ]:
x

In [ ]:
y

In [ ]:
#Split the data into 80% training and 20% testing 
x_train,  x_test, y_train, y_test = train_test_split(x, y, test_size= 0.2,random_state=42)

In [ ]:
print("---------------------------------------------------------")
print("Shape of training data: ", x_train.shape)
print("Shape of test data    : ", x_test.shape )
print("---------------------------------------------------------")

In [ ]:
x_scaler = StandardScaler()
x_scaler.fit(x)
column_names = x.columns
x[column_names] = x_scaler.transform(x)

In [ ]:
x.shape[1]

In [ ]:
#Build The model

model = Sequential()
model.add(Dense(128, input_dim=len(x.columns),kernel_initializer=k.initializers.random_normal(seed=13), activation="relu"))
model.add(Dense(64, kernel_initializer=k.initializers.random_normal(seed=13), activation="relu"))
model.add(Dense(32, kernel_initializer=k.initializers.random_normal(seed=13), activation="relu"))
model.add(Dense(1, activation="hard_sigmoid"))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.summary()

In [ ]:
from tensorflow.keras.utils import plot_model
from IPython.display import Image
plot_model(model, to_file='convnet.png', show_shapes=True,show_layer_names=True)
Image(filename='convnet.png')

In [ ]:
model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
lrd = ReduceLROnPlateau(monitor = 'val_loss',
                         patience = 20,
                         verbose = 1,
                         factor = 0.75,
                         min_lr = 1e-10)

mcp = ModelCheckpoint('model.h5')

es = EarlyStopping(verbose=1, patience=20)

In [ ]:
history = model.fit(x=x_train, y=y_train, epochs=800, callbacks=[lrd, mcp, es],batch_size=256, validation_split=0.1)

In [ ]:
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

# # summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper right')
plt.show()

In [ ]:
#Visualize the models accuracy and loss
plt.plot(history.history["accuracy"])
plt.plot(history.history["loss"])
plt.title("model accuracy & loss")
plt.ylabel("accuracy and loss")
plt.xlabel("epoch")
plt.legend(['acc', 'loss'], loc='lower right')
plt.show()

In [ ]:
for model_file in glob.glob('model.h5'):
  print("Model file: ", model_file)
  model = load_model(model_file)
  pred = model.predict(x_test)
  pred = [1 if y>=0.5 else 0 for y in pred] #Threshold, transforming probabilities to either 0 or 1 depending if the probability is below or above 0.5
  scores = model.evaluate(x_test, y_test)
  print()
  print("Original  : {0}".format(", ".join([str(x) for x in y_test])))
  print()
    
    
    
  print("Predicted : {0}".format(", ".join([str(x) for x in pred])))
  print() 
  print("Scores    : loss = ", scores[0], " acc = ", scores[1])
  print("---------------------------------------------------------")
  print()

In [ ]:
#wandb.log({'accuracy':x_test, 'loss':x_test})

In [ ]:
#wandb.log({'accuracy': x_train, 'loss': x_train})

In [ ]:
#wandb.init(project="Chronic Kidney Disease")

In [ ]:
#https://wandb.ai/yasirhessein/Chronic%20Kidney%20Disease?workspace=user-yasirhessein

### Good Luck